This notebook reads in the PHANGS CO map images and recasts them in deprojected space. Then it calculates the distance from the SNe to the nearest CO 2-1 emission. 

In [1]:
#import packages and initialize settings
import os
import numpy as np
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table
from astropy.io import ascii

plt.rc('text',usetex=True)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

import sys
sys.path.append('/home/mayker.1/Desktop/NessPythonFunctions')
from deprojectGalaxy import deproject

In [2]:
# Unpack master table to get SNe information
database = Table.read('../Data/3.SNe+GalData.csv', format = "csv")


In [3]:
# #OPEN CANDIDATE CATALOG AND READ IN RAS, DECS, NAMES
# galaxy = 'NGC4303'
# sne = 'SN1926A'
# image  = '../ALMAmaps/12m+7m+tp_co21_broad_mom0/NGC4303_12m+7m+tp_co21_broad_mom0.fits'
# errimage  = '../ALMAmaps/12m+7m+tp_co21_broad_emom0/NGC4303_12m+7m+tp_co21_broad_emom0.fits'
# alphaCOimg = '../alphaCOmaps/NGC4303_alphaCO21_Sun+20.fits'
# centerCoord = [185.47888, 4.473744]
# # (dist in kpc)
# incl, pa, galDist  = 20.0*u.deg, 310.6*u.deg, 17.6
# SNra, SNdec = 185.48041666666668*u.deg , 4.487972222222222*u.deg
# SNeCoord = [185.48041666666668 , 4.487972222222222]


# Galaxy, SNe = [],[]
# Galaxy.append(galaxy)
# SNe.append(sne)

In [13]:

#Stuff I wrote but haven't used in the refactor

from progressbar import ProgressBar

pbar = ProgressBar()

with open('SNeCOMasterCat7m.txt', 'w') as text_file:
    print('#Galaxy,Supernova,Ra,Dec,xCoord,yCoord,Type,numSNe,Intensity7m,IntensityErr7m,EW(7m),beamSize', file = text_file)

for i in pbar(range(len(galaxies))):
    found_name, found_type, found_ra, found_dec, x_coord, y_coord, map, map2,map3, naxis1, naxis2, beamSize = check_in_image(ras, decs, SNe, types, images[i], errFiles[i], ewFiles[i])
    plt.figure(figsize=(7, 7))
    logmap = np.log10(map)
    plt.imshow(map, origin='lower', interpolation='nearest', zorder=1)
    intensity = []
    intensityErr = []
    equivalentWidth = []
    equivalentWidthErr = []
    for k in range(len(found_name)):
        stringPrint = found_name[k] + ': ' + found_type[k]
        stringPrint = found_name[k]
        if (len(found_type[k])>1):
            if found_type[k][1] == "I":
                colorCode = 'chartreuse'
            elif found_type[k][1] == "b" or found_type[k][1] == "c":
                colorCode = 'salmon'
            elif found_type[k][1] == "a":
                colorCode = 'deeppink'
            else:
                colorCode = 'cyan'
        else:
            colorCode = 'slategrey'
        xVal = int(round(x_coord[k]))
        yVal = int(round(y_coord[k]))
        if (yVal <= naxis2) and (xVal <= naxis1):
            intensity.append(map[yVal, xVal])
            intensityErr.append(map2[yVal, xVal])
            equivalentWidth.append(map3[yVal, xVal])

        else: 
            intensity.append(np.nan)
            intensityErr.append(np.nan)
            equivalentWidth.append(np.nan)
        plt.plot(x_coord[k], y_coord[k], '*', ms=15, color=colorCode, zorder=10)
        plt.text(x_coord[k]-1, y_coord[k]+4, stringPrint, fontsize=14, fontweight='bold', color=colorCode)
    plt.axis('off')
    plt.title(galaxies[i], fontsize=24, fontweight='bold')
    plt.tight_layout()
    plt.savefig(galaxies[i] + '.png')
    plt.close()
    with open('SNeCOMasterCat7m.txt', 'a') as text_file:
        for j in range(len(found_name)):
            print(galaxies[i], ",", found_name[j],",", found_ra[j], ",",found_dec[j],",", x_coord[j],",",y_coord[j],",", found_type[j],",",len(found_name),",", intensity[j],",", intensityErr[j],",",equivalentWidth[j],",",beamSize, file = text_file)


NameError: name 'galaxies' is not defined

pull data value at x * y coord. error files are also available to go pull the same pixel to find uncertainty and add to the table.
check intensity values from ds9 but use puython to get the calue
error files are emom0
can check read me to see if I can find this stuff.

hdulist[0].data

In [ ]:
galaxies = np.loadtxt('PHANGSGalaxies_12_7_tp.txt', dtype = str, unpack=True, usecols = 0)
images = np.loadtxt('PHANGSGalaxiesImg_12_7_tp.txt', dtype = str, unpack=True, usecols = 0)
errFiles = np.loadtxt('PHANGSGalaxiesImgErr_12_7_tp.txt', dtype = str, unpack=True, usecols = 0)
ewFiles = np.loadtxt('PHANGSGalaxiesEW_12_7_tp.txt', dtype = str, unpack=True, usecols = 0)
pbar = ProgressBar()


with open('SNeCOMasterCat_12_7_tp.txt', 'w') as text_file:
    print('#Galaxy,Supernova,Ra,Dec,xCoord,yCoord,Type,numSNe,Intensity_12_7_tp,IntensityErr_12_7_tp,EW_12_7_tp', file = text_file)

for i in  pbar(range(len(galaxies))):
    found_name, found_type, found_ra, found_dec, x_coord, y_coord, map, map2,map3, naxis1, naxis2, beamSize = check_in_image(ras, decs, SNe, types, images[i], errFiles[i], ewFiles[i])
    plt.figure(figsize=(7, 7))
    #logmap = np.log10(map)
    plt.imshow(map, origin='lower', interpolation='nearest', zorder=1)
    intensity = []
    intensityErr = []
    equivalentWidth = []
    equivalentWidthErr = []
    for k in range(len(found_name)):
        stringPrint = found_name[k] + ': ' + found_type[k]
        stringPrint = found_name[k]
        if (len(found_type[k])>1):
            if found_type[k][1] == "I":
                colorCode = 'black'
            elif found_type[k][1] == "b" or found_type[k][1] == "c":
                colorCode = 'crimson'
            elif found_type[k][1] == "a":
                colorCode = 'purple'
            else:
                colorCode = 'cyan'
        else:
            colorCode = 'blue'
        xVal = int(round(x_coord[k]))
        yVal = int(round(y_coord[k]))
        if (yVal <= naxis2) and (xVal <= naxis1):
            intensity.append(map[yVal, xVal])
            intensityErr.append(map2[yVal, xVal])
            equivalentWidth.append(map3[yVal, xVal])

        else: 
            intensity.append(np.nan)
            intensityErr.append(np.nan)
            equivalentWidth.append(map3[yVal, xVal])
        plt.plot(x_coord[k], y_coord[k], '*', ms=15, color=colorCode, zorder=10)
        plt.text(x_coord[k]-1, y_coord[k]+4, stringPrint, fontsize=14, fontweight='bold', color=colorCode)
    plt.axis('off')
    plt.title(galaxies[i], fontsize=28, fontweight='bold')
    plt.tight_layout()
    plt.savefig(galaxies[i] + '.png')
    plt.close()
    with open('SNeCOMasterCat_12_7_tp.txt', 'a') as text_file:
        for j in range(len(found_name)):
            print(galaxies[i], ",", found_name[j],",", found_ra[j], ",",found_dec[j],",", x_coord[j],",",y_coord[j],",", found_type[j],",",len(found_name),",", intensity[j],",", intensityErr[j],",",equivalentWidth[j], file = text_file)


In [ ]:
# makeFlatHist takes in an image file and returns a sorted array of 
# each intensity value in map. This function removes NANs and turns
# negative intensity values into zeroes
def makeFlatHist(imageFile):
    hdulist = pyfits.open(imageFile)
    map = hdulist[0].data
    intensities = np.ndarray.flatten(map)
    sortInt = np.sort(intensities)
    idx = np.isfinite(sortInt)
    galData = sortInt[idx]
    #galData[galData < 0.0] = 0
    return(galData)
   
# findIntensity takes the intensity of the SN and matches it with 
# the last time that intensity occurs in the sorted array 
def findIntensity(SNInt, galInts):
    galZeroes, galVals = [],[]
    lowestNoise = galInts[0]
    #set all noise terms = 0 (noise is lowest value reflected around zero)
    #sort into zeroes array and intensity values array
    for i in range(len(galInts)):
        if np.fabs(galInts[i]) <= np.fabs(lowestNoise):
            galInts[i] = 0.0
            galZeroes.append(0.0)
        else:
            galVals.append(galInts[i])
    #Find percentile
    found = None
    if SNInt <= np.fabs(lowestNoise):
        foundProb = 0.0
    else:
        for i in range(len(galVals)):
            if galVals[i] == SNInt:
                found = i
        prob = np.linspace(0.0,1.0,len(galVals))
        foundProb = prob[found]
    return(galZeroes, galVals, lowestNoise, foundProb)

In [ ]:
imgFile = "12m+7m+tp_co21_broad_mom0/ngc4303_12m+7m+tp_co21_broad_mom0.fits"
SNInt = 5.85764611457112
SNName = "SN1999gn"

sortArray = makeFlatHist(imgFile)
zeroes, vals, noise, probability = findIntensity(SNInt, sortArray)

print(SNName, probability)

In [ ]:
SNeName, GalaxyName, Intensity, SNType = np.genfromtxt('SNeCOMasterCat_12_7_tp.txt', dtype = str, delimiter = ' , ', unpack=True, usecols = (1,0,8,6))

fltInten = [float(i) for i in Intensity]

Sne, galName, inten, sneType, GalaxyFile = [],[],[],[],[]

#Remove SNe from sample that aren't in map coverage
for i in range(len(SNeName)):
    if np.isfinite(fltInten[i]):
        Sne.append(SNeName[i])
        galName.append(GalaxyName[i])
        inten.append(fltInten[i])
        sneType.append(SNType[i])

#create list of galaxy file names
for i in range(len(galName)):
    if galName[i] == 'circinus':
        string = "12m+7m+tp_co21_broad_mom0/circinus_7m+tp_co21_broad_mom0.fits"
    elif galName[i] == 'ngc0253':
        string = '12m+7m+tp_co21_broad_mom0/ngc0253_7m+tp_co21_broad_mom0.fits'
    elif galName[i] == 'ngc2775':     
        string = '12m+7m+tp_co21_broad_mom0/ngc2775_12m+7m_co21_broad_mom0.fits'
    else:
        string = '12m+7m+tp_co21_broad_mom0/'+galName[i]+'_12m+7m+tp_co21_broad_mom0.fits'
    GalaxyFile.append(string)
    
percentile, medianInten, maxInten, noiseArray = [],[],[],[]

for i in range(len(Sne)):
    sortArray   = makeFlatHist(GalaxyFile[i])
    zeroes, vals, noise, probability = findIntensity(inten[i], sortArray)
    percentile.append(probability)
    noiseArray.append(noise)
    medianInten.append(np.median(vals)) 
    maxInten.append(sortArray[-1])
    
with open('CDF_Vals.txt', 'w') as text_file:
    print('#Galaxy, SNe, Type, Percentile, Intensity, MaxIntensity, Noise', file=text_file)
    for j in range(len(Sne)):
        print(galName[j], ' ', Sne[j], ' ', sneType[j], ' ', percentile[j], ' ', inten[j], maxInten[j], noiseArray[j], file = text_file)

In [ ]:
sortPerc = np.sort(percentile)
xvals = np.linspace(0,1,len(percentile))
maxRatio, medRatio = [],[]
#These should be arrays
for i in range(len(sortPerc)):
    maxRatio.append(inten[i]/maxInten[i])
    medRatio.append(inten[i]/medianInten[i])

maxRatio = np.sort(maxRatio)
medRatio = np.sort(medRatio)

unknown, sortTypeII, sortTypeIa, sortTypeSE = [],[],[],[]
uknMaxRat, TypeIIMaxRat, TypeIaMaxRat, TypeSEMaxRat = [],[],[],[]
uknMedRat, TypeIIMedRat, TypeIaMedRat, TypeSEMedRat = [],[],[],[]

# Type I and 'no' sorted as unknown
for i in range(len(sneType)):
    if (len(sneType[i]) == 1 or sneType[i] == 'no'):
        unknown.append(sortPerc[i])
        uknMaxRat.append(maxRatio[i])
        uknMedRat.append(medRatio[i])
    elif sneType[i][1] == "I":
        sortTypeII.append(sortPerc[i])
        TypeIIMaxRat.append(maxRatio[i])
        TypeIIMedRat.append(medRatio[i])
    elif sneType[i][1] == "a":
        sortTypeIa.append(sortPerc[i])
        TypeIaMaxRat.append(maxRatio[i])
        TypeIaMedRat.append(medRatio[i])
    elif sneType[i][1] == "b" or sneType[i][1] == "c":
        sortTypeSE.append(sortPerc[i]) 
        TypeSEMaxRat.append(maxRatio[i])
        TypeSEMedRat.append(medRatio[i])
            
xvalsII = np.linspace(0,1,len(sortTypeII))
xvalsIa = np.linspace(0,1,len(sortTypeIa))
xvalsSE = np.linspace(0,1,len(sortTypeSE))  
xvalsNo = np.linspace(0,1,len(unknown))    
print(len(sortPerc), len(sortTypeII), len(sortTypeIa),len(sortTypeSE),len(unknown))
    
fig = plt.figure(figsize = (7,5))
plt.tick_params(axis='both', direction='in')
plt.plot(sortPerc, xvals, ls='-', marker='o', color='black', label = 'All')
plt.plot(sortTypeII, xvalsII, ls='-.', marker='*',ms=9, color='purple', label = 'Type II')
plt.plot(sortTypeIa, xvalsIa, ls=':', marker='^', color = 'chocolate', label = 'Type Ia')
plt.plot(sortTypeSE, xvalsSE, ls='-', marker='o',color='green',label = 'Type Ib/c')
plt.plot(unknown, xvalsNo, ls='-', marker='<',label = 'Untyped')
plt.axhline(y=0.16, xmin=0, xmax = 1, ls = '--', color = 'k', alpha = 0.3)
plt.axhline(y=0.50, xmin=0, xmax = 1, ls = '--', color = 'k', alpha = 0.3)
plt.axhline(y=0.84, xmin=0, xmax = 1, ls = '--', color = 'k', alpha = 0.3)
plt.grid('both','both',color='k',alpha = 0.05, linestyle='-', linewidth=2)
plt.ylabel('Fraction of SNe', fontsize = 20)
plt.xlabel('Host Galaxy CO 2-1 Intensity Fraction', fontsize = 20)
plt.legend(loc='best', fontsize = 20)
fig.savefig('COIntCDF.png')

fig = plt.figure(figsize = (7,5))
plt.tick_params(axis='both', direction='in')
plt.plot(maxRatio, xvals, ls='-', marker='o', color='black', label = 'All')
plt.plot(TypeIIMaxRat, xvalsII, ls='-.', marker='*',ms=9, color='purple', label = 'Type II')
plt.plot(TypeIaMaxRat, xvalsIa, ls=':', marker='^', color = 'chocolate', label = 'Type Ia')
plt.plot(TypeSEMaxRat, xvalsSE, ls='-', marker='o',color='green',label = 'Type Ib/c')
plt.plot(uknMaxRat, xvalsNo, ls='-', marker='<',label = 'Untyped')
plt.axhline(y=0.16, xmin=0, xmax = 1, ls = '--', color = 'k', alpha = 0.3)
plt.axhline(y=0.50, xmin=0, xmax = 1, ls = '--', color = 'k', alpha = 0.3)
plt.axhline(y=0.84, xmin=0, xmax = 1, ls = '--', color = 'k', alpha = 0.3)
plt.grid('both','both',color='k',alpha = 0.05, linestyle='-', linewidth=2)
plt.ylabel('Fraction of SNe', fontsize = 20)
plt.xlabel('Host Galaxy CO 2-1 Intensity / Maximum Fraction', fontsize = 20)
plt.legend(loc='best', fontsize = 20)
fig.savefig('MaxRatioCDF.png')

fig = plt.figure(figsize = (7,5))
plt.tick_params(axis='both', direction='in')
plt.plot(medRatio, xvals, ls='-', marker='o', color='black', label = 'All')
plt.plot(TypeIIMedRat, xvalsII, ls='-.', marker='*',ms=9, color='purple', label = 'Type II')
plt.plot(TypeIaMedRat, xvalsIa, ls=':', marker='^', color = 'chocolate', label = 'Type Ia')
plt.plot(TypeSEMedRat, xvalsSE, ls='-', marker='o',color='green',label = 'Type Ib/c')
plt.plot(uknMedRat, xvalsNo, ls='-', marker='<',label = 'Untyped')
plt.axhline(y=0.16, xmin=0, xmax = 1, ls = '--', color = 'k', alpha = 0.3)
plt.axhline(y=0.50, xmin=0, xmax = 1, ls = '--', color = 'k', alpha = 0.3)
plt.axhline(y=0.84, xmin=0, xmax = 1, ls = '--', color = 'k', alpha = 0.3)
plt.grid('both','both',color='k',alpha = 0.05, linestyle='-', linewidth=2)
plt.ylabel('Fraction of SNe', fontsize = 20)
plt.xlabel('Host Galaxy CO 2-1 Intensity / Median Fraction', fontsize = 20)
plt.legend(loc='best', fontsize = 20)
fig.savefig('MedRatioCDF.png')